# Dependencies

In [1]:
from sys import path
path.append('../src/')

import cv2
import pickle as pkl
from os.path import join, exists
from glob import glob
from sys import stdout

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from icao_dataset import ICAODataset
from utils import load
from data_structures import Point, Eye, Rect
from global_config import FOLDER_IMAGES, FOLDER_MRKS, FILE_DATASET

In [2]:
PATH_TO_ROOT = ".."

FOLDER_IMAGES = join(PATH_TO_ROOT, FOLDER_IMAGES)
FOLDER_MRKS = join(PATH_TO_ROOT, FOLDER_MRKS)
FILE_DATASET = join(PATH_TO_ROOT, FILE_DATASET)

print(FOLDER_IMAGES)
print(FOLDER_MRKS)
print(FILE_DATASET)

..\data/cropped_faces_eyes/images/
..\data/cropped_faces_eyes/ground_truth/
..\data/cropped_faces_eyes/dataset_dlib.pkl


# Preprocessing 

In [3]:
def normalize_eye_position(eye, width, height):
    p = Point(width, height)
    return Eye.from_left_and_right_corners(
        left_corner=eye.left_corner / p,
        right_corner=eye.right_corner / p,
    )

# Normalized Ground-Truth

In [4]:
if exists(FILE_DATASET):
    print("Dataset found. Loading... ", end="")
    dataset = pkl.load(open(FILE_DATASET, "rb"))
    print("Done.")
else:
    print("Generating dataset...")
    dataset = ICAODataset()
    dataset.load(FOLDER_IMAGES, FOLDER_MRKS, output_size=(160, 160))
    pkl.dump(dataset, open(FILE_DATASET, "wb"))

Dataset found. Loading... Done.


In [5]:
N_FILES = len(dataset.image_files)

for i, (file, img, mrk) in enumerate(dataset):
    print(f"{i + 1} of {N_FILES}", end="\r")
    stdout.flush()
    
    height, width, _ = img.shape
    mrk.left_eye = normalize_eye_position(mrk.left_eye, width, height)
    mrk.right_eye = normalize_eye_position(mrk.right_eye, width, height)

In [6]:
FILE_NORMALIZED = FILE_DATASET.replace(".pkl", "_normalized.pkl")
pkl.dump(dataset, open(FILE_NORMALIZED, "wb"))

In [7]:
print(FILE_NORMALIZED)

..\data/cropped_faces_eyes/dataset_dlib_normalized.pkl
